In [34]:
pip install pdfplumber pytesseract pdf2image Pillow pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import pdfplumber
import re
import json
import pandas as pd
import os
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Rahul\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
print("Libraries imported")

Libraries imported


In [36]:
import importlib
import blood_report_parser
importlib.reload(blood_report_parser)
from blood_report_parser import extract_text, extract_parameters, validate
print("Module loaded")

Module loaded


In [39]:
report_path = "C:\\Users\\Rahul\\Desktop\\Project\\Health_Diagnosis\\datasets\\sterling-accuris-pathology-sample-report-unlocked.pdf"

if os.path.exists(report_path):
    raw_text = extract_text(report_path)
    print("Extracted text (first 500 chars):")
    print(raw_text[:500])
else:
    print("File not found")

Extracted text (first 500 chars):
MC-2202
Scan QR code to check
report authenticity
Passport No : LABORATORY TEST REPORT
Patient Information Sample Information Client/Location Information
Name : Lyubochka Svetka Lab Id : 02232160XXXX Client Name : Sterling Accuris Buddy
Registration on : 20-Feb-2023 09:10
Location :
Sex/Age : Male / 41 Y 01-Feb-1982 Collected at : non SAWPL
Approved on : 20-Feb-2023 11:09 Status : Final
Ref. Id : Collected on : 20-Feb-2023 08:53
Printed On : 28-Feb-2023 10:26
Ref. By : Sample Type : EDTA Blood
P


In [40]:
params = extract_parameters(raw_text)
print("Extracted Parameters:")
print(params)

Extracted Parameters:
{'hemoglobin': 14.5, 'glucose': 157.07}


In [41]:
validated_params = validate(params)
print("Validated Parameters:")
for param, value in validated_params.items():
    print(f"{param}: {value}")

Validated Parameters:
hemoglobin: 14.5
glucose: 157.07


In [42]:
with open("parameter_ranges.json", 'r') as f:
    ranges = json.load(f)
print("Ranges loaded:")
print(ranges)

Ranges loaded:
{'hemoglobin': {'low': 12.0, 'high': 16.0}, 'wbc': {'low': 4.0, 'high': 11.0}, 'rbc': {'low': 4.5, 'high': 5.5}, 'platelets': {'low': 150.0, 'high': 400.0}, 'glucose': {'low': 70.0, 'high': 140.0}}


In [43]:
from model1_parameter_interpreter import classify

classification_results = {}
for param, value in validated_params.items():
    if value is not None and param in ranges:
        status = classify(param, value, ranges)
        classification_results[param] = {"value": value, "status": status}
        print(f"{param}: {value} - {status}")

hemoglobin: 14.5 - Normal
glucose: 157.07 - High


In [44]:
df_data = []
for param, info in classification_results.items():
    df_data.append({'Parameter': param, 'Value': info['value'], 'Status': info['status']})

df = pd.DataFrame(df_data)
df.to_csv("blood_parameters.csv", index=False)
print(df)
print("Saved to blood_parameters.csv")

    Parameter   Value  Status
0  hemoglobin   14.50  Normal
1     glucose  157.07    High
Saved to blood_parameters.csv
